In [1]:
import os
import shutil
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow import keras
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

class_names = ['0', '1', '2', '3', 
               '4', '5', '6', '7', 
               '8', '9', 'A', 'B', 
               'C', 'D', 'E', 'F', 
               'G', 'H', 'K', 'L', 
               'M', 'N', 'O', 'P', 
               'R', 'S', 'T', 'U', 
               'V', 'X', 'Y', 'Z']

datagen = ImageDataGenerator(
    rescale=1.0/255, 
    validation_split=0.2,
)

train_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='training',
    class_mode='sparse'
)

validation_generator = datagen.flow_from_directory(
    "../Training_set/charTrainset", 
    target_size=(28,12), 
    batch_size=20,
    subset='validation',
    class_mode='sparse'
)

Found 1729 images belonging to 32 classes.
Found 417 images belonging to 32 classes.


In [2]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(128, (5,5), activation='elu', 
#                            padding='same',
#                            input_shape=(28, 12, 3)),
#     tf.keras.layers.Conv2D(128, (5,5), activation='elu',
#                            padding='same'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Conv2D(256, (3,3), activation='elu', 
#                            padding='same'),
#     tf.keras.layers.Conv2D(256, (3,3), activation='elu',
#                            padding='same'),
#     tf.keras.layers.Conv2D(128, (3,3), activation='elu',
#                            padding='same'),
#     tf.keras.layers.MaxPooling2D(2, 2),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='elu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(128, activation='elu'),
#     tf.keras.layers.Dropout(0.4),
#     tf.keras.layers.Dense(32, activation='softmax')
# ])
model = tf.keras.models.load_model('../Model/Model2')

2022-01-08 08:22:21.710657: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-01-08 08:22:21.710943: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.
2022-01-08 08:22:21.766185: W tensorflow/core/graph/graph_constructor.cc:761] Node 'StatefulPartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 5 outputs. Output shapes may be inaccurate.
2022-01-08 08:22:21.778863: W tensorflow/core/graph/graph_constructor.cc:761] Node 'StatefulPartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 4 outputs. Output shapes may be inaccurate.
2022-01-08 08:22

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 12, 128)       9728      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 12, 128)       409728    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 6, 128)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 6, 256)        295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 6, 256)        590080    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 14, 6, 128)        295040    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 3, 128)         0

In [4]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [5]:
# class stopCallback(tf.keras.callbacks.Callback):
#     def on_epoch_end(self, epoch, logs={}) :
#         if (logs.get('accuracy') > 0.99) :
#             print('\nReached 99% accuracy so stopping training')
#             self.model.stop_training = True

In [6]:
# callback = stopCallback()
# model.fit(train_generator, epochs=100, callbacks=[callback])

In [7]:
# model.evaluate(validation_generator, verbose=1)

In [12]:
path1 = "../Testing_set/greenparkingchar"
path2 = "../Testing_set/greenparking2char"

all_path = [path1, path2]

destination = "../charLabel/"
trash = "../charLabel/ZTrash"

for path in all_path:
    for file in os.listdir(path) :
        image_mp= mpimg.imread(path + '/' + file) / 255
        image_mp = np.expand_dims(image_mp, axis=0)
        prediction = model.predict(image_mp)
        if (np.max(prediction) < 0.8) :
            if (np.max(prediction) < 0.65) : 
                shutil.move(path + '/' + file, trash)
                continue
        shutil.move(path + '/' + file, destination + class_names[np.argmax(prediction)] + '/' + file)

In [ ]:
# model.save('../Model/Model2')